In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [20]:
# 1. LOAD DATA
# Loading the dataset from the previous cleaning stage
df = pd.read_csv('../data/processed/preprocessing5.csv')

In [21]:
# Numeric cleanup (handling method strings and NaNs)
cols_to_fix = ['pl_orbpererr1', 'pl_orbpererr2', 'pl_orbsmaxerr1', 'pl_orbsmaxerr2']
for col in cols_to_fix:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

In [22]:
def calculate_advanced_habitability(row):
    # 1. Radius Score (Ideal: 1.0 Re)
    # sigma=0.5 allows Super-Earths to stay in the high-score range
    s_rad = np.exp(-0.5 * ((row['pl_rade'] - 1.0) / 0.5)**2)
    
    # 2. Insolation Score (Ideal: 1.0 Earth Flux)
    # We use log10 because energy flux varies massively
    s_flux = np.exp(-0.5 * ((np.log10(row['pl_insol'] + 1e-5) - 0) / 0.4)**2)
    
    # 3. Thermal Score (Ideal: 288K)
    s_temp = np.exp(-0.5 * ((row['pl_eqt'] - 288.0) / 70.0)**2)
    
    # 4. Stability Score (Ideal: 0.0 Eccentricity)
    # A circular orbit is 1.0, highly elliptical is 0.0
    s_eccen = 1.0 - row['pl_orbeccen']
    
    # 5. Compositional Safety (Hard Penalty for Gas Giants)
    # Allows a "soft landing" up to 1.6, then drops off
    penalty = 1.0 if row['pl_rade'] <= 1.6 else np.exp(-0.5 * (row['pl_rade'] - 1.6))
    
    # CALCULATE FINAL SCORE (Geometric Mean for higher accuracy)
    # This reflects the multi-column contribution you asked for
    final_score = (s_rad * s_flux * s_temp * s_eccen)**(1/4) * penalty
    
    return final_score

# Apply the new high-accuracy logic
df['habitability_score'] = df.apply(calculate_advanced_habitability, axis=1)

# Verification
top_candidates = df.sort_values(by='habitability_score', ascending=False)
print(f"Planets with Score > 0.7: {len(top_candidates[top_candidates['habitability_score'] > 0.7])}")
print(top_candidates[['pl_name', 'habitability_score']].head(26))

Planets with Score > 0.7: 31
            pl_name  habitability_score
5166   TRAPPIST-1 d            0.973924
5115      TOI-700 e            0.969919
3565   Kepler-438 b            0.968993
5114      TOI-700 d            0.941963
3487   Kepler-395 c            0.933864
5167   TRAPPIST-1 e            0.932365
4320     LP 890-9 c            0.927455
2522  Kepler-1649 c            0.915411
1428        K2-72 e            0.910817
2521  Kepler-1649 b            0.903206
4779     TOI-2095 c            0.894666
3570   Kepler-442 b            0.892754
1426        K2-72 c            0.868299
1266         K2-3 d            0.855666
5165   TRAPPIST-1 c            0.841830
2791   Kepler-186 e            0.822784
2527  Kepler-1652 b            0.821399
3825    Kepler-62 e            0.813724
244     Gliese 12 b            0.797308
5121      TOI-715 b            0.779370
2440  Kepler-1582 b            0.762199
5168   TRAPPIST-1 f            0.754248
3226   Kepler-296 e            0.735391
3738   Kepl

C:\Users\hardi\AppData\Local\Temp\ipykernel_23288\1811471335.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['habitability_score'] = df.apply(calculate_advanced_habitability, axis=1)


In [23]:
# 3. FEATURE SELECTION: The "Detective" Approach
# To prevent cheating, we drop the 'answers' (pl_rade, pl_eqt)
leaky_cols = ['pl_eqt', 'pl_rade', 'habitability_score']
X_prelim = df.select_dtypes(include=[np.number]).drop(columns=[c for c in leaky_cols if c in df.columns])
y = df['habitability_score']

# Using 200 estimators for higher stability/accuracy
rf_selector = RandomForestRegressor(n_estimators=200, random_state=42)
rf_selector.fit(X_prelim, y)

# Filter Top 50 Features (The scientific clues)
importances = pd.DataFrame({'feature': X_prelim.columns, 'importance': rf_selector.feature_importances_})
top_50_features = importances[~importances['feature'].str.contains('err|lim')].sort_values(by='importance', ascending=False).head(50)

In [24]:
pillars = ['pl_rade', 'pl_insol', 'pl_eqt', 'pl_orbeccen']
identifiers = ['pl_name', 'hostname','st_spectype', 'discoverymethod']
final_cols = identifiers + top_50_features['feature'].tolist() + pillars + ['habitability_score']
df_final = df[final_cols].copy()

In [26]:
# 5. NORMALIZATION
exclude_from_scaling = ['pl_name', 'hostname', 'habitability_score', 'st_spectype', 'discoverymethod']
cols_to_scale = df_final.select_dtypes(include=[np.number]).columns
cols_to_scale = [c for c in cols_to_scale if c not in exclude_from_scaling]

scaler = StandardScaler()
df_scaled = df_final.copy()
df_scaled[cols_to_scale] = scaler.fit_transform(df_scaled[cols_to_scale])


df_scaled.to_csv('final-preprocessed-updated.csv', index=False)

In [27]:
df_scaled.loc[df_scaled['habitability_score']>=0.7, ['pl_name', 'habitability_score']].shape

(31, 2)